## ETL Notebook for CA Biositing Project

This notebook provides a documented walkthrough of the ETL (Extract, Transform, Load) process for the CA Biositing project. It is designed for interactive development and exploration before migrating logic into the production pipeline.

It covers:

1.  **Setup**: Importing necessary libraries and establishing a connection to the database.
2.  **Extraction**: Pulling raw data from Google Sheets.
3.  **Cleaning**: Standardizing data types, handling missing values, and cleaning column names.
4.  **Normalization**: Replacing human-readable names (e.g., "Corn") with database foreign key IDs (e.g., `resource_id: 1`).
5.  **Utilities**: Common functions for data manipulation and analysis.
6.  **Deployment Plan**: A step-by-step guide for moving the code from this notebook into the production ETL modules.

In [14]:
import os
import sys
import pandas as pd
import numpy as np
import janitor as jn
import logging
from IPython.display import display
from sqlalchemy.orm import Session
from sqlalchemy import select

# --- Basic Logging Configuration for Notebook ---
# When running in a notebook, we use Python's standard logging.
# In the production pipeline, this will be replaced by Prefect's `get_run_logger()`
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# --- Robustly find the project root ---
# This ensures that the notebook can be run from any directory within the project.
path = os.getcwd()
project_root = None
while path != os.path.dirname(path):
    if 'pixi.toml' in os.listdir(path):
        project_root = path
        break
    path = os.path.dirname(path)

if not project_root:
    raise FileNotFoundError("Could not find project root containing 'pixi.toml'.")

# Add the project root to the Python path to allow for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    logger.info(f"Added project root '{project_root}' to sys.path")
else:
    logger.info(f"Project root '{project_root}' is already in sys.path")

# --- Import project modules ---
try:
    from src.ca_biositing.pipeline.ca_biositing.pipeline.utils.engine import engine
    from src.ca_biositing.datamodels.ca_biositing.datamodels.schemas.generated.ca_biositing import *
    from src.ca_biositing.pipeline.ca_biositing.pipeline.utils.name_id_swap import replace_name_with_id_df
    from src.ca_biositing.pipeline.ca_biositing.pipeline.etl.extract import proximate, ultimate, cmpana
    logger.info('Successfully imported all project modules.')
except ImportError as e:
    logger.error(f'Failed to import project modules: {e}', exc_info=True)

2026-01-04 08:53:57,422 - INFO - Project root '/Users/pjsmitty301/ca-biositing' is already in sys.path
2026-01-04 08:53:57,423 - INFO - Successfully imported all project modules.


### Data Cleaning Function

In [18]:
def clean_the_gsheets(df):
    """Cleans and standardizes a DataFrame extracted from Google Sheets.

    This function performs several key operations:
    1. Cleans column names to a standard format (snake_case).
    2. Drops rows where essential columns ('repl_no', 'value') are empty.
    3. Coerces data types for numeric and datetime columns, handling errors gracefully.
    4. Converts remaining columns to the best possible data types.

    Args:
        df (pd.DataFrame): The raw DataFrame.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    logger.info('Starting DataFrame cleaning process.')
    if not isinstance(df, pd.DataFrame):
        logger.error('Input is not a pandas DataFrame.')
        return None
    
    try:
        # 1. Clean names and drop rows with missing essential data
        df_cleaned = df.clean_names().dropna(subset=['repl_no', 'value'])
        logger.info(f'Dropped {len(df) - len(df_cleaned)} rows with missing values.')

        # 2. Coerce numeric types
        df_cleaned['repl_no'] = pd.to_numeric(df_cleaned['repl_no'], errors='coerce').astype('Int32')
        df_cleaned['value'] = pd.to_numeric(df_cleaned['value'], errors='coerce').astype(np.float32)

        # 3. Coerce datetime types
        if 'created_at' in df_cleaned.columns:
            df_cleaned['created_at'] = pd.to_datetime(df_cleaned['created_at'], errors='coerce')
        if 'updated_at' in df_cleaned.columns:
            df_cleaned['updated_at'] = pd.to_datetime(df_cleaned['updated_at'], errors='coerce')

        # 4. Convert other dtypes to best possible
        df_cleaned = df_cleaned.convert_dtypes()
        logger.info('Successfully cleaned DataFrame.')
      
    
        # 5. Convert all string data to lowercase
        df_cleaned = df_cleaned.applymap(lambda s: s.lower() if isinstance(s, str) else s)
        logger.info('Converted all string data to lowercase.')
        return df_cleaned

    except Exception as e:
        logger.error(f'An error occurred during DataFrame cleaning: {e}', exc_info=True)
        return None

        

### Data Normalization Function

In [19]:
def normalize_dataframes(dataframes, normalize_columns):
    """Normalizes a list of DataFrames by replacing name columns with foreign key IDs.

    This function iterates through a list of dataframes and, for each one, iterates
    through a dictionary of columns that need to be normalized. It uses the 
    `replace_name_with_id_df` utility to look up or create the corresponding ID
    in the database.

    Args:
        dataframes (list[pd.DataFrame]): A list of DataFrames to normalize.
        normalize_columns (dict): A dictionary mapping column names to SQLModel classes and attributes.

    Returns:
        list[pd.DataFrame]: The list of normalized DataFrames.
    """
    logger.info(f'Starting normalization process for {len(dataframes)} dataframes.')
    normalized_dfs = []
    try:
        with Session(engine) as db:
            for i, df in enumerate(dataframes):
                if not isinstance(df, pd.DataFrame):
                    logger.warning(f'Item {i+1} is not a DataFrame, skipping.')
                    continue
                
                logger.info(f'Processing DataFrame #{i+1} with {len(df)} rows.')
                df_normalized = df.copy()

                for df_col, (model, model_name_attr) in normalize_columns.items():
                    if df_col not in df_normalized.columns:
                        logger.warning(f"Column '{df_col}' not in DataFrame #{i+1}. Skipping normalization for this column.")
                        continue
                    
                    try:
                        logger.info(f"Normalizing column '{df_col}' using model '{model.__name__}'.")
                        df_normalized = replace_name_with_id_df(
                            db=db,
                            df=df_normalized,
                            ref_model=model,
                            df_name_column=df_col,
                            model_name_attr=model_name_attr,
                            id_column_name='id',
                            final_column_name=f'{df_col}_id'
                        )
                        new_col_name = f'{df_col}_id'
                        num_nulls = df_normalized[new_col_name].isnull().sum()
                        logger.info(f"Successfully normalized '{df_col}'. New column '{new_col_name}' contains {num_nulls} null values.")
                    except Exception as e:
                        logger.error(f"Error normalizing column '{df_col}' in DataFrame #{i+1}: {e}", exc_info=True)
                        continue # Continue to the next column
                
                normalized_dfs.append(df_normalized)
                logger.info(f'Finished processing DataFrame #{i+1}.')
            
            logger.info('Committing database session.')
            db.commit()
            logger.info('Database commit successful.')
    except Exception as e:
        logger.error(f'A critical error occurred during the database session: {e}', exc_info=True)
        db.rollback()
        logger.info('Database session rolled back.')
        
    return normalized_dfs

### ETL Execution Example

In [20]:
# --- 1. Extraction ---
# In a real Prefect flow, each extraction would be a separate task.
logger.info('Starting data extraction...')
prox_df = proximate.extract(project_root=project_root)
ult_df = ultimate.extract(project_root=project_root)
cmp_df = cmpana.extract(project_root=project_root)
dataframes = [prox_df, ult_df, cmp_df]
logger.info('Data extraction complete.')

# --- 2. Cleaning ---
# This list comprehension applies the cleaning function to each extracted dataframe.
logger.info('Starting data cleaning...')
clean_dataframes = [clean_the_gsheets(df) for df in dataframes if df is not None]
logger.info('Data cleaning complete.')

# --- 3. Normalization ---
# This dictionary defines the columns to be normalized. 
# The key is the column name in the DataFrame.
# The value is a tuple containing the corresponding SQLAlchemy model and the name of the attribute on the model to match against.
NORMALIZE_COLUMNS = {
    'resource': (Resource, 'name'),
    'prepared_sample': (PreparedSample, 'name'),
    'preparation_method': (PreparationMethod, 'name'),
    'parameter': (Parameter, 'name'),
    'unit': (Unit, 'name'),
    'analyst_email': (Contact, 'email'),
    'analysis_type': (AnalysisType, 'name'),
    'primary_ag_product': (PrimaryAgProduct, 'name')
}

logger.info('Starting data normalization...')
normalized_dataframes = normalize_dataframes(clean_dataframes, NORMALIZE_COLUMNS)
logger.info('Data normalization complete.')

# --- 4. Display Results ---
logger.info('Displaying results of normalization...')
for i, df in enumerate(normalized_dataframes):
    print(f'--- Normalized DataFrame {i+1} ---')
    display(df.head())

2026-01-04 08:58:52,849 - INFO - Starting data extraction...
2026-01-04 08:58:52,865 - INFO - HTTP Request: GET http://127.0.0.1:4200/api/admin/version "HTTP/1.1 200 OK"
2026-01-04 08:58:52,877 - INFO - Extracting raw data from '03.1-Proximate' in 'Aim 1-Feedstock Collection and Processing Data-BioCirV'...
2026-01-04 08:58:54,699 - INFO - Successfully extracted raw data.
2026-01-04 08:58:54,703 - INFO - Finished in state Completed()
2026-01-04 08:58:54,724 - INFO - HTTP Request: GET http://127.0.0.1:4200/api/admin/version "HTTP/1.1 200 OK"
2026-01-04 08:58:54,736 - INFO - Extracting raw data from '03.7-Ultimate' in 'Aim 1-Feedstock Collection and Processing Data-BioCirV'...
2026-01-04 08:58:55,992 - INFO - Successfully extracted raw data.
2026-01-04 08:58:55,996 - INFO - Finished in state Completed()
2026-01-04 08:58:56,016 - INFO - HTTP Request: GET http://127.0.0.1:4200/api/admin/version "HTTP/1.1 200 OK"
2026-01-04 08:58:56,029 - INFO - Extracting raw data from '03.3-CmpAna' in 'Aim

--- Normalized DataFrame 1 ---


,prox_uuid_031,record_id,source_codename,storage_cond,exper_abbrev,repl_no,repl_id,value,created_at,updated_at,qc_result,upload_status,note,resource_id,prepared_sample_id,preparation_method_id,parameter_id,unit_id,analyst_email_id,analysis_type_id
0,d7965110-407f-e356-d41d-b3b9a2b7b7,(73)b7b7,oakleaf,4c,prox01xk,1.0,prox01xk(73)1,61.849998,2024-10-02 10:31:01,NaT,pass,not ready,,24,109,7,25,1,1,6
1,c8fea984-2e9a-8def-55fb-1a9d7d9ba8,(73)9ba8,oakleaf,4c,prox01xk,2.0,prox01xk(73)2,63.209999,2024-10-02 10:31:31,NaT,pass,ready,,24,109,7,25,1,1,6
2,df304d5d-3a85-4881-7142-6d4e5f957d,(73)957d,oakleaf,4c,prox01xk,3.0,prox01xk(73)3,63.270000,2024-10-02 10:32:01,NaT,pass,imported,,24,109,7,25,1,1,6
3,01c6c5be-cea6-54af-3924-b0bad69335,(73)9335,oakleaf,4c,prox01xk,1.0,prox01xk(73)1,0.690000,2024-10-03 10:31:01,NaT,pass,import failed,,24,109,7,23,1,1,6
4,126745c7-dd41-2f6d-0dc5-28dbca415f,(73)415f,oakleaf,4c,prox01xk,2.0,prox01xk(73)2,0.890000,2024-10-03 10:31:31,NaT,pass,,,24,109,7,23,1,1,6


--- Normalized DataFrame 2 ---


,ult_uuid_037,record_id,ult_sample_name,storage_cond,exper_abbrev,repl_no,repl_id,value,created_at,updated_at,...,equipment,raw_data_url,upload_status,resource_id,prepared_sample_id,preparation_method_id,parameter_id,unit_id,analyst_email_id,analysis_type_id
0,421617a5-e50e-9642-2974-b3275fe822,)u22e822,hum-almhu023km2(15)u22,rt vacuum sealed,ult26kh,1,ult26kh(15)1,88.599998,NaT,NaT,...,,,,8,80,6,28,3,2,7
1,7e7919c2-5db4-6bef-75e2-7e51321200,)u001200,hum-almhu023km2(15)u00,rt vacuum sealed,ult26kh,1,ult26kh(15)1,NaN,NaT,NaT,...,,,,8,80,6,28,3,2,7
2,3aa85881-1185-642f-c44b-41ad2275d2,)ud275d2,hum-almsh022km2(13)ud2,rt vacuum sealed,ult26kh,1,ult26kh(13)1,81.300003,NaT,NaT,...,,,,26,96,6,28,3,2,7
3,fa418804-6c4f-4c90-d78f-84d7df54d3,)ud354d3,ene-wash017okm2(82)ud3,rt vacuum sealed,ult26kh,1,ult26kh(82)1,92.199997,NaT,NaT,...,,,,2,91,9,28,3,2,7
4,6fdacbfc-7e0b-473b-444f-85b7650267,)u670267,ebo-gppm010okm2(1b)u67,rt vacuum sealed,ult26kh,1,ult26kh(1b)1,93.800003,NaT,NaT,...,,,,7,153,9,28,3,2,7


--- Normalized DataFrame 3 ---


,cmp_uuid_033,record_id,storage_cond,exper_abbrev,repl_no,repl_id,value,created_at,updated_at,qc_result,...,equipment,raw_data_url,upload_status,resource_id,prepared_sample_id,preparation_method_id,parameter_id,unit_id,analyst_email_id,analysis_type_id
0,3ee2993d-86e3-1f16-c7ea-f8d555e114,(85)e114,rt vacuum sealed,cmp04xk,1,cmp04xk(85)1,14.16,2025-01-23 09:00:01,NaT,pass,...,,,ready,24,92,8,12,2,1,3
1,46878ef9-1226-22a0-d5d8-cf65e241cb,(85)41cb,rt vacuum sealed,cmp04xk,2,cmp04xk(85)2,14.18,2025-01-23 09:00:16,NaT,pass,...,,,ready,24,92,8,12,2,1,3
2,76a7a2f4-c4e4-e60f-1187-dec6e02246,(85)2246,rt vacuum sealed,cmp04xk,3,cmp04xk(85)3,14.12,2025-01-23 09:00:31,NaT,pass,...,,,ready,24,92,8,12,2,1,3
3,7a136832-286b-07cb-62de-acf52f9311,(85)9311,rt vacuum sealed,cmp04xk,1,cmp04xk(85)1,15.74,2025-01-23 09:00:46,NaT,pass,...,,,ready,24,92,8,14,2,1,3
4,b709ecee-f9a6-a55d-a59e-93b7b863d7,(85)63d7,rt vacuum sealed,cmp04xk,2,cmp04xk(85)2,15.75,2025-01-23 09:01:01,NaT,pass,...,,,ready,24,92,8,14,2,1,3


### Deployment Plan

The code in this notebook will be transitioned to the main ETL pipeline by following these steps:

1.  **Function Migration**: The `clean_the_gsheets` and `normalize_dataframes` functions will be moved to a new utility module, for example, `src/ca_biositing/pipeline/ca_biositing/pipeline/utils/etl_utils.py`. Each function will be decorated with `@task` from Prefect to turn it into a reusable pipeline component.
2.  **Flow Creation**: A new Prefect flow will be created in the `src/ca_biositing/pipeline/ca_biositing/pipeline/flows/` directory (e.g., `master_extraction_flow.py`). This flow will orchestrate the entire ETL process for a given data source.
3.  **Task Integration**: The new flow will be composed of individual tasks. It will call the existing extraction tasks (`proximate.extract`, etc.), and then pass the results to the new cleaning and normalization tasks from `etl_utils.py`.
4.  **Logging**: The `logging` module will be replaced with `get_run_logger()` from Prefect within the tasks to ensure logs are captured by the Prefect UI.
5.  **Configuration**: The `NORMALIZE_COLUMNS` dictionary will be moved to a configuration file or defined within the relevant flow to make it easier to manage and modify without changing the code.
6.  **Testing**: Unit tests will be written for the new utility functions in `etl_utils.py`. An integration test will be created for the new Prefect flow to ensure all the tasks work together correctly.
7.  **Deployment**: Once the flow is complete and tested, it will be deployed to the Prefect server using the `pixi run deploy` command, making it available to be run on a schedule or manually via the UI.